In [ ]:
pip install numpy pandas nltk tensorflow keras


In [14]:
import json
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk

nltk.download('punkt')

# Load dataset
with open('/content/mental-health (1).json') as file:
    data = json.load(file)

# Extract patterns and tags
patterns, tags = [], []
for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])

# Encode tags
le = LabelEncoder()
tags_encoded = le.fit_transform(tags)

# Tokenize patterns
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(patterns)
sequences = tokenizer.texts_to_sequences(patterns)
padded_sequences = pad_sequences(sequences, padding='post')

vocab_size = len(tokenizer.word_index) + 1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 16, input_length=padded_sequences.shape[1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, tags_encoded, epochs=100, verbose=1)


In [19]:
def chat():
    print("Chatbot is ready! hello,tell me how you are feeling.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break
        seq = tokenizer.texts_to_sequences([user_input])
        padded = pad_sequences(seq, maxlen=padded_sequences.shape[1], padding='post')
        prediction = model.predict(padded)
        tag = le.inverse_transform([np.argmax(prediction)])
        for intent in data['intents']:
            if intent['tag'] == tag[0]:
                print("Chatbot:", np.random.choice(intent['responses']))
                break

chat()


Chatbot is ready! hello,tell me how you are feeling.
You: hello


NameError: name 'model' is not defined

In [ ]:
import json
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
import tensorflow as tf # Import tensorflow here
import os # Import os for file path operations

nltk.download('punkt')

# Load dataset
with open('/content/mental-health (1).json') as file:
    data = json.load(file)

# ... (rest of your data loading and preprocessing code) ...

def chat():
    print("Chatbot is ready! hello,tell me how you are feeling.")

    # Check if model file exists
    model_file_path = 'mental_health_bot_model.h5'
    if os.path.exists(model_file_path):
        # Load the saved model
        model = tf.keras.models.load_model(model_file_path)
    else:
        # Define and train the model if it doesn't exist
        model = tf.keras.Sequential([
            tf.keras.layers.Embedding(vocab_size, 16, input_length=padded_sequences.shape[1]),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(len(le.classes_), activation='softmax')
        ])
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(padded_sequences, tags_encoded, epochs=100, verbose=0)  # Train the model
        # Save the trained model
        model.save(model_file_path)

    while True:
        # This block needs to be indented to be inside the while loop
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break
        seq = tokenizer.texts_to_sequences([user_input])
        padded = pad_sequences(seq, maxlen=padded_sequences.shape[1], padding='post')
        prediction = model.predict(padded)
        tag = le.inverse_transform([np.argmax(prediction)])
        for intent in data['intents']:
            if intent['tag'] == tag[0]:
                print("Chatbot:", np.random.choice(intent['responses']))
                break

chat()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Chatbot is ready! hello,tell me how you are feeling.


You: im sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 547ms/step
Chatbot: Hello there. Glad to see you're back. What's going on in your world right now?


In [ ]:
# Save the model and tokenizer
model.save('mental_health_bot_model.h5')

import pickle
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(le, file)

# To reload later:
# model = tf.keras.models.load_model('mental_health_bot_model.h5')
# with open('tokenizer.pkl', 'rb') as file:
#     tokenizer = pickle.load(file)
# with open('label_encoder.pkl', 'rb') as file:
#     le = pickle.load(file)
